In [1]:
import numpy as np
import pandas as pd

In [2]:
from scipy.spatial.distance import squareform, pdist

In [3]:
from scipy.sparse.csgraph import minimum_spanning_tree

In [4]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt

In [5]:
import sys

In [6]:
from sklearn.metrics.pairwise import pairwise_distances

In [7]:
from sklearn.metrics import f1_score

In [48]:
from sklearn.metrics.cluster import fowlkes_mallows_score

In [69]:
from sklearn.neighbors import KNeighborsClassifier

In [100]:
def labelClusters2(clusters, target, data):
    mySet = set(clusters)
    resultList = []
    for i in mySet:
        myPos = []
        for j in range(len(clusters)):
            if (clusters[j] == i):
                myPos.append(j)
        
        myData = []
        myTarget = []
        for k in range(len(myPos)):
            myData.append(data[j])
            myTarget.append(target[j])
        
        centroid = []
        for l in range(len(myData[0])):
            mySum = 0
            for m in range(len(myData)):
                mySum += myData[m][l]
            centroid.append(mySum/len(myData))
        
        if (len(myData) > 4):
            neigh = KNeighborsClassifier(n_neighbors=5)
        else:
            neigh = KNeighborsClassifier(n_neighbors=1)
            
        neigh.fit(myData, myTarget)
        neigh.predict(centroid)
        resultList.append(neigh.predict(centroid)[0])
    
    mySetArray = []
    for i in mySet:
        mySetArray.append(i)
        
    labels = []
        
    for i in range(len(clusters)):
        for j in range(len(mySetArray)):
            if (clusters[i] == mySetArray[j]):
                labels.append(resultList[j])
    return labels

In [137]:
def labelClusters(clusters, target):
    mySet = set(clusters)
    resultList = []
    for i in mySet:
        myClasses = []
        for j in range(len(clusters)):
            if (clusters[j] == i):
                myClasses.append(target[j])
        resultList.append(max(set(myClasses), key=myClasses.count))
    
    mySetArray = []
    for i in mySet:
        mySetArray.append(i)
        
    labels = []  
    for i in range(len(clusters)):
        for j in range(len(mySetArray)):
            if (clusters[i] == mySetArray[j]):
                labels.append(resultList[j])
    return labels

In [172]:
abs(-2)

2

In [173]:
def gini2(arr):
    
    somai = 0
    somatot = arr[-1]
    for i in range(len(arr)-1):
        somaj = 0
        for j in range(i+1, len(arr)):
            somaj += abs(arr[i] - arr[j])
        somai += somaj
        somatot += arr[i]
    mult = (len(arr) - 1) * somatot
    
    return (somai/mult)

In [10]:
def counting(arr):
    mySet = set(arr)
    myList = []
    myNewSet = []
    for i in mySet:
        myNewSet.append(i)
        
    for i in range(len(myNewSet)):
        myList.append(0)
        for j in arr:
            if (myNewSet[i]==j):
                myList[i] += 1
    return myList

In [251]:
menorCluster([0,9,0,9,0,9,0])

[1, 3, 5]

In [250]:
def menorCluster(clusters):
    mySet = set(clusters)
    minimo = len(clusters)
    
    for i in mySet:
        if((clusters.count(i)) < minimo):
            minimo = clusters.count(i)
            
    myList = []
    
    for j in range(len(clusters)):
        if(clusters.count(clusters[j])==minimo):
            myList.append(j)
            
    return myList

In [237]:
def genieLinkage2(data, limiar, clusterNumber):
    
    clusters = {}
    row_index = 0
    col_index = 0
    array = []
    

    for n in range(data.shape[0]):
        array.append(n)
    
        
    clusters[0] = array.copy()
    
    dists = []
    
    for i in range(0, data.shape[0]):
        for j in range(i, data.shape[1]):
            dists.append(([data[i][j], i, j]))
    
    dists.sort(key=lambda x: x[0])
    pos = 0

    for k in range(1, data.shape[0]):
        min_val = sys.maxsize
        
        if(gini2(counting(clusters[k-1])) <= limiar):
            for i in range(pos, len(dists)):
                if(dists[i][0] == data[dists[i][1]][dists[i][2]]):
                    pos = i
                    min_val = dists[i][0]
                    row_index = dists[i][1]
                    col_index = dists[i][2]
                    break
        else:
            iList = menorCluster(clusters[k-1])
            for i in iList:
                for j in range(0, data.shape[0]):
                    if(data[i][j]<min_val):
                        min_val = data[i][j]
                        row_index = i
                        col_index = j

        array = clusters[k-1].copy()
        minimum = min(array[row_index],array[col_index])
        maximum = max(array[row_index],array[col_index])
        
        for n in range(len(array)):
            if(array[n]==maximum):
                array[n] = minimum
        clusters[k] = array.copy()
        
        for i in range(len(array)):
            for j in range(len(array)):
                if(array[i]==array[j]):
                    data[i][j] = sys.maxsize
                    
        if (len(set(clusters[k])) == clusterNumber):
            return clusters[k]

In [117]:
def genie2(dataset, target, limiar, clusterNumber):
    
#     dataX = dataset.loc[:,dataset.columns[:-1]].values
#     dataY = dataset.loc[:,[dataset.columns[-1]]].values
    dataX = dataset.values
    dataY = target[0]
    
    dataDistances = pairwise_distances(dataX,metric='euclidean')
    #making all the diagonal elements infinity 
    np.fill_diagonal(dataDistances,sys.maxsize)
    
    clusters = genieLinkage2(dataDistances, limiar, clusterNumber)
    
    labels = labelClusters(clusters, dataY)
    #labels = labelClusters2(clusters,dataY, dataX)
    
    #fmScore = f1_score(dataY, labels, average='micro')
    fmScore = fowlkes_mallows_score(dataY, labels)
    
    return fmScore

In [16]:
a1 = pd.read_table("datasets/a1.data", header=None, delim_whitespace=True)
iris5 = pd.read_table("datasets/iris5.data", header=None, delim_whitespace=True)
flame = pd.read_table("datasets/flame.data", header=None, delim_whitespace=True)
pathbased = pd.read_table("datasets/pathbased.data", header=None, delim_whitespace=True)
unbalance = pd.read_table("datasets/unbalance.data", header=None, delim_whitespace=True)

In [17]:
a1labels = pd.read_table("labels/a1.labels", header=None, delim_whitespace=True)
iris5labels = pd.read_table("labels/iris5.labels", header=None, delim_whitespace=True)
flamelabels = pd.read_table("labels/flame.labels", header=None, delim_whitespace=True)
pathbasedlabels = pd.read_table("labels/pathbased.labels", header=None, delim_whitespace=True)
unbalancelabels = pd.read_table("labels/unbalance.labels", header=None, delim_whitespace=True)

In [18]:
import time

In [238]:
t1 = time.time()
print(genie2(iris5, iris5labels, 0.3, 3))
t2 = time.time()
print(t2 - t1)

0.847263910606
0.3553299903869629


In [34]:
t1 = time.time()
print(genie2(pathbased, pathbasedlabels, 0.6, 3))
t2 = time.time()
print(t2 - t1)

0.75108849742
4.343164920806885


In [239]:
#0.691 ##0.665 0.738 0.765## 0.764 0.764 0.764 0.886 0.673
iris5data = []
for i in range(1,7):
    if (i==1):
        iris5data.append((genie2(iris5, iris5labels, 1, 3)))
    else:
        iris5data.append((genie2(iris5, iris5labels, i/10, 3)))
iris5data

[0.69054827106065841,
 0.84726391060604256,
 0.84726391060604256,
 0.84726391060604256,
 0.88574209259747094,
 0.67286625768733155]

In [240]:
#só executar da primeira vez

result = pd.DataFrame(columns = ["single","gini_02","gini_03","gini_04","gini_05","gini_06"])
result.loc[0] = iris5data
result = result.rename({0:'iris5'})
result.to_csv("results/result.csv")

In [241]:
def computeScore(base):
    result = pd.read_csv("results/result.csv", index_col=0)
    
    data = pd.read_table("datasets/"+base+".data", header=None, delim_whitespace=True)
    labels = pd.read_table("labels/"+base+".labels", header=None, delim_whitespace=True)
    k = len(set(labels[0]))
    
    resultData = []
    for i in range(1,7):
        if (i==1):
            resultData.append((genie2(data, labels, 1, k)))
        else:
            resultData.append((genie2(data, labels, i/10, k)))
    
    size = len(result)
    result.loc[size] = resultData
    result = result.rename({size:base})
    result.to_csv("results/result.csv")

In [242]:
computeScore("iris")
#computeScore("Aggregation")

In [243]:
computeScore("flame")

In [244]:
result = pd.read_csv("results/result.csv", index_col=0)
result

,single,gini_02,gini_03,gini_04,gini_05,gini_06
iris5,0.690548,0.847264,0.847264,0.847264,0.885742,0.672866
iris,0.763517,0.923434,0.923434,0.923434,0.753795,0.753795
flame,0.730024,1.000000,1.000000,1.000000,1.000000,1.000000


In [174]:
gini(counting(unbalancelabels[0]))

0.54807692307692313

In [175]:
gini2(counting(unbalancelabels[0]))

0.6263736263736264

In [187]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import fcluster